# ID2214 Assignment 2 Group no. 5
### Project members: 
[Ceren Dikmen, cerend@kth.se] [Jakob Heyder, heyder@kth.se] [Lutfi Altin, lutfia@kth.se] [Muhammad Fasih Ullah, mufu@kth.se]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.


### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above,
and thereby 


## Load NumPy, pandas and time

In [0]:
import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [0]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_normalization(df, normalizationtype = "minmax"):
    copydf = df.copy()
    normalization = {}
    
    cols = copydf.loc[:, ~df.columns.isin(['ID', 'CLASS'])] # Removing ID and CLASS columns
    cols = cols.select_dtypes(include=[np.int_, np.float_]) # Selecting only int and float type columns from the remaining
    
    if normalizationtype == 'minmax':
        for column in cols:
            min = copydf[column].min()
            max = copydf[column].max()
            copydf[column] = [(x-min)/(max-min) for x in copydf[column]]
            normalization[column] = ("minmax", min, max)
            
    elif normalizationtype == 'zscore':
        for column in cols:
            mean = copydf[column].mean()
            std = copydf[column].std()
            copydf[column] = copydf[column].apply(lambda x: (x-mean)/std)
            normalization[column] = ("zscore", mean, std)
            
    return copydf, normalization

def apply_normalization(df, normalization):
    copydf = df.copy()
    
    for column in normalization:
        type = normalization[column][0]
        
        if type == 'minmax':
            min = normalization[column][1]
            max = normalization[column][2]
            copydf[column] = np.clip([(x-min)/(max-min) for x in copydf[column]], 0, 1) # Clipping outliers to 0-1
            
        elif type == 'zscore':
            mean = normalization[column][1]
            std = normalization[column][2]
            copydf[column] = copydf[column].apply(lambda x: (x-mean)/std)
   
    return copydf


def create_imputation(df):
    copydf = df.copy()
    imputation = {}
    
    cols = copydf.loc[:, ~df.columns.isin(['ID', 'CLASS'])] # Dropping ID and CLASS columns
    numbercols = cols.select_dtypes(include=[np.int_, np.float_]) # Choosing int and float columns
    othercols = cols.select_dtypes(exclude=[np.int_, np.float_]) # Choosing all other columns 
    
    for column in numbercols:
        copydf[column].fillna(copydf[column].mean(),inplace=True) # For numeric columns replacing them with mean
        imputation[column] = copydf[column].mean()
        
        if copydf[column].isna().any(): # Check if whole column was NaN
            copydf[column].fillna(0, inplace=True) #Replace with 0
            imputation[column] = 0
        
    for column in othercols:
        # Mode returns an array. In order to get the correct mode we have to use iloc[0]
        copydf[column].fillna(copydf[column].mode().iloc[0],inplace=True)
        imputation[column] = copydf[column].mode().iloc[0]
        
        # Check if column still has some NA 
        # if it does, change it to category[to match the output] and replace it with the first category
        if copydf[column].isna().any():
          
            copydf[column] = copydf[column].astype('category')
            
            # [Old code, doesn't work] fill = "" if copydf[column].dtype == 'object' else df.cat.categories[0]
            
            fill = "" if copydf[column].dtype == 'object' else copydf[column].astype('category').cat.categories[0]
            copydf[column].fillna(fill, inplace=True)
            imputation[column] = fill
            
            # print("CopyDF: ", column, copydf[column].dtype, copydf[column].astype('category').cat.categories[0])
            
    return copydf, imputation

def apply_imputation(df, imputation):
    copydf = df.copy()
    
    for column in imputation:
        copydf[column].fillna(imputation[column], inplace=True)
    
    return copydf

def create_one_hot(df):
    copydf = df.copy()
    one_hot = {}
    
    # filter all columns except ID and CLASS
    cols = copydf.loc[:, ~df.columns.isin(['ID', 'CLASS'])]
    # filter only categorial data
    cols = cols.select_dtypes(include=["object", "category"])
    
    for col in cols:
        # convert to get categories
        if copydf[col].dtype == 'object':
            copydf[col] = copydf[col].astype('category')
            
        # convert categorial to binary (one-hot)    
        dummies = pd.get_dummies(copydf[col])

        # create binary column for each category
        one_hot[col] = {}
        for cat in dummies:
            one_hot[col][cat] = col + '-' + cat
            copydf[col + '-' + cat] = dummies[cat]
            # convert to float type (hint 4)
            copydf[col + '-' + cat] = copydf[col + '-' + cat].astype('float')
            
        # drop original columns     
        copydf.drop(col, axis=1, inplace=True)
    
    return copydf, one_hot

def apply_one_hot(df, one_hot):
    copydf = df.copy()
    
    for col in one_hot:
        # convert categorial to binary (one-hot)
        dummies = pd.get_dummies(copydf[col])
        # iterate over categories generated in one-hot
        for (cat, col_name) in one_hot[col].items():
            # for each category take dummy value
            copydf[col_name] = dummies[cat]
            # convert to float type (hint 4)
            copydf[col_name] = copydf[col_name].astype('float')
        copydf.drop(col, axis=1, inplace=True)
    
    return copydf

#################################
#                               #
# Performance measure functions #
#                               #
#################################

def accuracy(df, correctlabels):
    return sum(df.idxmax(axis=1)==correctlabels)/len(correctlabels)

def brier_score(df, correctlabels):
    score = 0
    
    # get observed probabilities (one for each correct label, otherwise zero)
    observed_probs = pd.get_dummies(correctlabels)
    # vectorized formula slides brier score (probability - observed_probability) squared
    score = (df - observed_probs) ** 2
    # sum over different labels, and sum all instances
    score = score.sum(axis=1)
    # average over instances
    score = score.mean()
        
    return score

# function for one label , returns tpr 
def auc_single(predictions, correctlabels, threshold, c):
   
    # array with true for correct labels for class c (by row index)
    correctlabels_class = np.array(correctlabels)==predictions.columns[c]
    
    # array with predictions for all instances that should be classified class c
    predictions_class = predictions[ predictions.columns[c] ]
    
    # array with true for all correctly predicted labels according to threshold
    predicted_labels = predictions_class[correctlabels_class] >= threshold
    pos = sum(predicted_labels)
    
    # correctly predicted instances (according to threshold) divided by total number of instances that should be class c
    tpr = pos / sum(correctlabels_class)
    
    # repeat for false positive rate (instances not in class)
    not_correctlabels_class = np.array(correctlabels)!=predictions.columns[c]
    predictions_class = predictions[ predictions.columns[c] ]
    predicted_labels = predictions_class[not_correctlabels_class] >= threshold
    neg = sum(predicted_labels)
    fpr = neg / sum(not_correctlabels_class)
    
    return tpr, fpr


def auc(predictions, correctlabels):
    thresholds = np.unique(predictions)
    total_number_of_labels = len(correctlabels)
    
    AUCs = {}
    
    # iterate over all classes and calculate the area under the ROC(tpr/fpr) curve (AUC)
    for (idx,c) in enumerate(np.unique(correctlabels)):
        single = [auc_single(predictions, correctlabels, t, idx) for t in reversed(thresholds)]
                    
        # calculate AUC as area under the curve
        AUC = 0
        tpr_last = 0
        fpr_last = 0
        
        # iterate over all thresholds
        for s in single:
            tpr, fpr = s
            
            # Case 1.) Add area under triangle        
            if tpr > tpr_last and fpr > fpr_last:
                AUC += (fpr-fpr_last)*tpr_last + (fpr-fpr_last)*(tpr-tpr_last) / 2
            
            # Case 2.) Add area under rectangle            
            elif fpr > fpr_last:
                AUC += (fpr-fpr_last)*tpr
            
            # update point coordinates (tpr, fpr) of curve
            tpr_last = tpr
            fpr_last = fpr
       
        AUCs[c] = AUC
        
                
    # take the weighted average for all classes (dependent on their frequency of occourance)
    AUC_total = 0
    for (cName,auc) in AUCs.items():
        number_of_labels = np.sum(np.array(correctlabels) == cName)
        weight = number_of_labels / total_number_of_labels
        AUC_total += weight * auc
        
    return AUC_total  

def create_bins(df, nobins=10, bintype="equal-width"):
    copydf = df.copy()
    binning = {}

    cols = copydf.loc[:, ~df.columns.isin(['ID', 'CLASS'])]
    numbercols = cols.select_dtypes(include=[np.int_, np.float_])

    for col in numbercols:
        cutfunc = pd.cut if bintype == "equal-width" else pd.qcut
        copydf[col], bins = cutfunc(copydf[col], nobins, labels=False, retbins=True, duplicates="drop")
        # hint 4-5
        copydf[col] = copydf[col].astype('category', categories=list(range(nobins)))

        # hint 6
        bins[0] = -np.inf
        bins[-1] = np.inf
        binning[col] = bins

    return copydf, binning


def apply_bins(df, binning):
    copydf = df.copy()

    for column in binning:
        copydf[column] = pd.cut(copydf[column], labels=False, bins=binning[column])

        # hint 3-4
        nobins = len(binning[column] - 1)  # n bins will generate n+1 values
        copydf[column] = copydf[column].astype('category', categories=list(range(nobins)))

    return copydf

## 1. Define the class kNN

In [0]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# imputation, normalization, one_hot, labels, training_labels, training_data
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype: "minmax" (default) or "zscore"
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.normalization should be a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot should be a one-hot mapping (see Assignment 1; can be excluded if this function was not completed)
# self.training_labels should be a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels should be the categories of the previous series
# self.training_data should be the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
# normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns 
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# k: an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are estimated by the relative class frequencies in the set of class labels from the k nearest 
#              (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation, normalization and (possibly) one-hot
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

class kNN():
    def __init__(self):
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None

    def fit(self, df, normalizationtype = 'minmax'):
        # assign mapping and take copy for next computation
        copydf_imp, self.imputation = create_imputation(df)
        copydf_norm, self.normalization = create_normalization(copydf_imp, normalizationtype)
        copydf_one_hot, self.one_hot = create_one_hot(copydf_norm)
        self.training_labels = copydf_one_hot["CLASS"].astype('category')
        self.training_data = np.array(copydf_one_hot[df.columns.difference(['CLASS', 'ID'])])
        
        
    def predict(self, df, k):
        # apply imputation, normalization and one_hot 
        copydf_imp = apply_imputation(df, self.imputation)
        copydf_norm = apply_normalization(copydf_imp, self.normalization)
        copydf_one_hot = apply_one_hot(copydf_norm, self.one_hot)
        unique_labels = np.unique(self.training_labels)
                
        # drop ID and CLASS columns
        cols = copydf_one_hot[df.columns.difference(['CLASS', 'ID'])]
        
        # get numerical values as ndarray
        numbercols = np.array(cols.select_dtypes(include=[np.int_, np.float_])) # Choosing int and float columns
        othercols = cols.select_dtypes(exclude=[np.int_, np.float_]) # Choosing all other columns 
        
        #creates a new dataframe that's zero-initialized with indicies and labels as columns
        resultdf = pd.DataFrame(0.0, index=range(0,len(self.training_data)), columns = unique_labels) 
        for (idx, row) in enumerate(numbercols):
            # calculate probability for each class (neighbours_in_class divided by k)
            class_probabilities = self.get_nearest_neighbor_predictions(row, k)
            # insert probabilities to dataframe
            for cProb in class_probabilities:
                resultdf.at[idx, int(cProb[0])] = cProb[1]
        return resultdf
            
    def get_nearest_neighbor_predictions(self, test_x, k):
        # calculate distance to all training samples
        distances = np.empty(self.training_data.shape[0])
        for (idx, train_x) in enumerate(self.training_data):
            distances[idx] = self.distance(train_x, test_x)
        # stack <idx> column to corresponding <distance>
        distances = np.column_stack((range(0, len(self.training_data)), distances))
        # sort by <distance> argument   
        sorted_indicies = distances[:,1].argsort()
        distances = distances[sorted_indicies]
        # take k-neighbours with lowest distance
        k_indicies = distances[:k, 0]
        k_labels = self.training_labels[k_indicies]
        # count label occourances and calculate probability
        unique, counts = np.unique(k_labels, return_counts=True)
        probabilities = np.column_stack((unique, counts / np.sum(counts)))
        return probabilities  

    def distance(self, x1, x2):
        # euclidean distance
         return np.linalg.norm(x1-x2)
            
        

In [0]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

results


Training time: 0.03 s.
Testing time (k=1): 0.21 s.
Testing time (k=3): 0.23 s.
Testing time (k=5): 0.21 s.
Testing time (k=7): 0.21 s.
Testing time (k=9): 0.21 s.


,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [0]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.2f}".format(brier_score(predictions,train_labels)))


Accuracy on training set (k=1): 1.00
AUC on training set (k=1): 1.00
Brier score on training set (k=1): 0.00


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [0]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.class_priors should be a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
# to the relative frequencies of the labels
# self.feature_class_value_counts should be a mapping from the feature (column name) to the number of
# training instances with a specific combination of (non-missing, categorical) value for the feature and class label
# self.feature_class_counts should me a mapping from the feature (column name) to the number of
# training instances with a specific class label and some (non-missing, categorical) value for the feature
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
# predictions with estimated class probabilities for each row in df, where the class probabilities
# are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply discretization
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors

class NaiveBayes:
    def __init__(self):
        self.binning = None
        self.class_priors = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None

    def fit(self, df, nobins=10, bintype="equal-width"):
        df_copy, binning = create_bins(df, nobins, bintype)
        cols = df_copy.loc[:, ~df_copy.columns.isin(['ID', 'CLASS'])]
        self.binning = binning
        self.feature_class_counts = dict()
        self.feature_class_value_counts = dict()
        self.class_priors = dict()

        count = df_copy.groupby('CLASS').count()
        for index, row in count.iterrows():
            self.feature_class_counts[index] = row[0]

        for column in cols:
            self.feature_class_value_counts[column] = dict()

        for key, value in self.feature_class_counts.items():
            filtered = df_copy.loc[df_copy['CLASS'] == key]
            for column in cols:
                count = filtered.groupby(column).count()
                self.feature_class_value_counts[column][key] = []
                for index, row in count.iterrows():
                    self.feature_class_value_counts[column][key].append(row[0])

        for key, value in self.feature_class_counts.items():
             self.class_priors[key] = value / sum(self.feature_class_counts.values())

    def predict(self, df):
        df_copy = apply_bins(df, self.binning)
        cols = df_copy.loc[:, ~df_copy.columns.isin(['ID', 'CLASS'])]
        df_return = pd.DataFrame(columns=list(self.feature_class_counts.keys()))
        for index, row in df_copy.iterrows():
            rel_freq = dict()
            class_prob = dict()
            for cname, ccount in self.feature_class_counts.items():
                rel_freq[cname] = dict()
                for col in cols:
                    fvalue = row[col]
                    fcount = self.feature_class_value_counts[col][cname][fvalue]
                    rel_freq[cname][col] = fcount / ccount

            for cname, cfreq in rel_freq.items():
                class_prob[cname] = self.class_priors[cname] * np.prod(list(cfreq.values()))
            prob_sum = sum(class_prob.values())
            for cname in class_prob.keys():
                if prob_sum == 0:
                    class_prob[cname] = self.class_priors[cname]
                else:
                    class_prob[cname] = class_prob[cname] / prob_sum
            df_return = df_return.append(pd.Series(class_prob), ignore_index=True)
        return df_return


In [0]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

results


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:238: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:256: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


Training time (3, 'equal-width'): 0.22 s.
Testing time (3, 'equal-width'): 0.24 s.
Training time (3, 'equal-size'): 0.23 s.
Testing time (3, 'equal-size'): 0.23 s.
Training time (5, 'equal-width'): 0.22 s.
Testing time (5, 'equal-width'): 0.24 s.
Training time (5, 'equal-size'): 0.23 s.
Testing time (5, 'equal-size'): 0.23 s.
Training time (10, 'equal-width'): 0.24 s.
Testing time (10, 'equal-width'): 0.23 s.
Training time (10, 'equal-size'): 0.24 s.
Testing time (10, 'equal-size'): 0.23 s.


Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [0]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:238: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:256: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


Accuracy on training set: 0.85
AUC on training set: 0.97
Brier score on training set: 0.23


### Comment on assumptions, things that do not work properly, etc.